In this project, I analyzed customer behavior for online retail store that sells unique all-occasion gift-ware in the UK.

The dataset consists of 1,067,371 transactions and has the following variables:

Variable	Description
InvoiceNo	Invoice number. Nominal. A 6-digit integral number uniquely assigned to each transaction. If this code starts with the letter 'c', it indicates a cancellation.
StockCode	Product (item) code. Nominal. A 5-digit integral number uniquely assigned to each distinct product.
Description	Product (item) name. Nominal.
Quantity	The quantities of each product (item) per transaction. Numeric.
InvoiceDate	Invice date and time. Numeric. The day and time when a transaction was generated.
UnitPrice	Unit price. Numeric. Product price per unit in sterling.
CustomerID	Customer number. Nominal. A 5-digit integral number uniquely assigned to each customer.
Country	Country name. Nominal. The name of the country where a customer resides.
I calculated three types of revenue-based CLV, assuming Average Lifespan for Basic and Granular CLV being 36 months:

Basic CLV = Average Revenue per Month * Average Lifespan

Granular CLV = (Average Revenue per Transaction * Average Frequency per Month) * Average Lifespan

Traditional CLV = Average Revenue * (Retention Rate / Churn Rate)



In [2]:
import pandas as pd
import numpy as np

Customer Lifetime Value (CLV) = how much a company expects to earn from an average customer in a life time.

CLV allows to benchmark customers and identify how much money the company can afford to spend on customer acquisition.
Historical CLV = the sum of revenues of all customer transactions multiplied by average or product-level profit margin

Problems with historical CLV:

doesn't account for customer tenure, retention or churn (e.g. if the company is growing its customer base, historical CLV will be deflated due to short tenure)
doesn't account for new customers and their future revenue

Basic CLV = Average Revenue Profit Margin Average Lifespan
(where Average Lifespan is e.g. average time before customer churn)

Granular CLV = (Average Revenue per Transaction Average Frequency Profit Margin) * Average Lifespan
(where Avg Frequency is within the certain timeframe, e.g. a month)
accounts for each transaction

Traditional CLV = (Average Revenue Profit Margin) Retention Rate / Churn Rate
(where Churn = 1 - Retention Rate)
Retention/Churn - a proxy of expected length of customer lifespan with the company
account for customer loyalty assumes that churn is final and customers do not return (especially critical for non-contractual business models)

But before calculating Basic, Granular and Traditional CLV, we will load, explore the data and then calculate retention rates using cohort analysis. We will need retention rates and churn rates for calculating Traditional CLV later.


Load and Explore Data

In [3]:
import xlrd
import pandas as pd
retail = pd.read_excel("C:\\Users\\Admin\\Desktop\\FILES\\AKPROJECTS\\Customer Segmentation\\online_retail.xlsx")

In [4]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [5]:
retail.shape

(525461, 8)

In [6]:
print("number of unique customers:", retail['Customer ID'].nunique())

number of unique customers: 4383


In [7]:
#checking duplicates
print(f"there are {retail.duplicated().sum()} duplicated rows")

there are 6865 duplicated rows


In [8]:
#viewing duplicated rows
retail[retail.duplicated()].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
371,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
383,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329.0,United Kingdom
384,489517,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,12,2009-12-01 11:34:00,0.65,16329.0,United Kingdom
385,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
386,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
390,489517,84951A,S/4 PISTACHIO LOVEBIRD COASTERS,1,2009-12-01 11:34:00,2.55,16329.0,United Kingdom
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
394,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
657,489529,22028,PENNY FARTHING BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom
658,489529,22036,DINOSAUR BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom


In [9]:
retail = retail.drop_duplicates(keep='first')

In [10]:

#checking missing values
retail.isna().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107833
Country             0
dtype: int64

In [11]:

retail = retail[retail['Customer ID'].notna()]

In [12]:
# checking types of columns
retail.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object

Calculating Cohort Index

In [13]:

retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

In [14]:
retail['YearMonth'] = retail['InvoiceDate'].dt.strftime("%Y-%m")
retail['YearMonth'] = pd.to_datetime(retail['YearMonth'])

In [17]:
retail['CohortMonth'] = retail.groupby(by=['Customer ID'])['YearMonth'].transform('min')

In [22]:

def calculate_cohort_index(df, transactions_col, CohortMonth):
    
    year_diff = df[transactions_col].dt.year - df[CohortMonth].dt.year

    month_diff = df[transactions_col].dt.month - df[CohortMonth].dt.month
    
    cohort_index = year_diff * 12 + month_diff + 1
    
    return cohort_index

In [23]:
retail['CohortIndex'] = calculate_cohort_index(retail, 'InvoiceDate', 'CohortMonth')

In [24]:

retail['CohortIndex'].max()

13

In [25]:
retail.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,YearMonth,CohortMonth,CohortIndex
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,2010-12-01,2009-12-01,13
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,2010-12-01,2009-12-01,13
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,2010-12-01,2009-12-01,13
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,2010-12-01,2009-12-01,13
525460,538171,21931,JUMBO STORAGE BAG SUKI,2,2010-12-09 20:01:00,1.95,17530.0,United Kingdom,2010-12-01,2009-12-01,13


 Calculating Average Active Users per Cohort

In [26]:
users_in_cohorts = retail.groupby(by=['CohortMonth', 'CohortIndex'])['Customer ID'].nunique()

In [27]:
users_in_cohorts = users_in_cohorts.reset_index()
users_in_cohorts.head()

,CohortMonth,CohortIndex,Customer ID
0,2009-12-01,1,1045
1,2009-12-01,2,392
2,2009-12-01,3,358
3,2009-12-01,4,447
4,2009-12-01,5,410


In [28]:

cohorts_fin = users_in_cohorts.pivot(index='CohortMonth', columns='CohortIndex', values='Customer ID')
cohorts_fin

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2009-12-01,1045.0,392.0,358.0,447.0,410.0,408.0,408.0,374.0,355.0,392.0,452.0,518.0,260.0
2010-01-01,394.0,86.0,119.0,120.0,110.0,115.0,105.0,91.0,114.0,134.0,122.0,37.0,NaN
2010-02-01,363.0,109.0,82.0,110.0,93.0,76.0,79.0,103.0,100.0,106.0,32.0,NaN,NaN
2010-03-01,436.0,95.0,113.0,103.0,100.0,87.0,105.0,130.0,126.0,36.0,NaN,NaN,NaN
2010-04-01,291.0,67.0,58.0,47.0,54.0,67.0,79.0,76.0,22.0,NaN,NaN,NaN,NaN
2010-05-01,254.0,49.0,45.0,49.0,48.0,66.0,56.0,22.0,NaN,NaN,NaN,NaN,NaN
2010-06-01,269.0,58.0,53.0,55.0,62.0,76.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,183.0,38.0,37.0,52.0,55.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,158.0,39.0,50.0,51.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Calculating Retention Rates

In [29]:
cohort_sizes = cohorts_fin.iloc[:, 0]
retention = cohorts_fin.divide(cohort_sizes, axis='rows').round(3)

In [30]:
retention

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2009-12-01,1.0,0.375,0.343,0.428,0.392,0.390,0.390,0.358,0.340,0.375,0.433,0.496,0.249
2010-01-01,1.0,0.218,0.302,0.305,0.279,0.292,0.266,0.231,0.289,0.340,0.310,0.094,NaN
2010-02-01,1.0,0.300,0.226,0.303,0.256,0.209,0.218,0.284,0.275,0.292,0.088,NaN,NaN
2010-03-01,1.0,0.218,0.259,0.236,0.229,0.200,0.241,0.298,0.289,0.083,NaN,NaN,NaN
2010-04-01,1.0,0.230,0.199,0.162,0.186,0.230,0.271,0.261,0.076,NaN,NaN,NaN,NaN
2010-05-01,1.0,0.193,0.177,0.193,0.189,0.260,0.220,0.087,NaN,NaN,NaN,NaN,NaN
2010-06-01,1.0,0.216,0.197,0.204,0.230,0.283,0.074,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,1.0,0.208,0.202,0.284,0.301,0.115,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,1.0,0.247,0.316,0.323,0.127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Calculate Basic CLV (revenue-based)

Basic CLV = Average Revenue * Average Lifespan

In [31]:
retail['PurchaseSize'] = retail['Quantity']*retail['Price']

In [32]:
# calculate monthly spend per customer
monthly_revenue = retail.groupby(by=['Customer ID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

596.4745756097561

Average Lifespan can be inferred by looking into the average time it takes for customers to churn from the time they made their first purchase. 
For now, we assume that the customer lifespan is 3 years or 36 months.

In [33]:
lifespan_months = 36

In [34]:
CLV_basic = monthly_revenue * lifespan_months
CLV_basic

21473.08472195122

Calculate Granular CLV (revenue-based)
Granular CLV = (Average Revenue per Transaction Average Frequency) Average Lifespan

In [35]:
# first mean() averages the revenue per invoice 
# second mean() averages the revenue per purchase
revenue_per_purchase = retail.groupby(by=['Invoice'])['PurchaseSize'].mean().mean()
revenue_per_purchase

28.78630387308043

In [36]:
# calculate average number of unique invoices per customer each month
avg_frequency_per_month = retail.groupby(by=['Customer ID', 'YearMonth'])['Invoice'].nunique().mean()
avg_frequency_per_month

1.6920373027259685

In [37]:
CLV_granular = revenue_per_purchase * avg_frequency_per_month * lifespan_months
CLV_granular

1753.469998590856

Calculate Traditional CLV (revenue-based)

Traditional CLV method doens't require to define customer lifespan and instead uses retention to churn rate to access customer life expectancy.

Traditional CLV = Average Revenue * Retention Rate / Churn Rate

In [38]:
# calculate monthly spend per customer
monthly_revenue = retail.groupby(by=['Customer ID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

596.4745756097561

In [39]:

#calculate average retention rate per customer
avg_retention = retention.iloc[:, 1:].mean().mean()
avg_retention

0.2553991372053872

In [40]:

avg_churn = 1 - avg_retention
avg_churn

0.7446008627946128

In [41]:
CLV_traditional = monthly_revenue * (avg_retention/avg_churn)
CLV_traditional

204.5916135578018

In [42]:
print("Average CLV: {:.2f} USD at {:.1%} average retention rate and {:.2f} USD average monthly revenue".format(CLV_traditional, avg_retention, monthly_revenue))

Average CLV: 204.59 USD at 25.5% average retention rate and 596.47 USD average monthly revenue
